In [1]:
import json
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import numpy as np
import matplotlib.pyplot as plt
from names_dataset import NameDataset
from tqdm import tqdm
from spacy.tokenizer import Tokenizer
import spacy
from spacy.lang.en import English
nlp = English()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)
# nlp = spacy.load("en_core_web_sm")

### Pre-Note
Token labels are presented in BIO (Beginning, Inner, Outer) format. The PII type is prefixed with “B-” when it is the beginning of an entity. If the token is a continuation of an entity, it is prefixed with “I-”. Tokens that are not PII are labeled “O”.

**Model idea** Lets instead of using the predefined token space, we define a new one without BIO definition, but where we concat tokens that are part of the same entity. We can then afterwards use SpaCy tokenizer to reverse the changes we made and get the labels.

In [2]:
file_path = "../data/train.json"

with open(file_path, "r") as file:
    data = json.load(file)

df = pd.DataFrame(data)

In [3]:
df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [4]:
labels = df.labels.explode().reset_index(drop=True)
labels_vc = labels.value_counts()
labels_vc

labels
O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

In [5]:
df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [6]:
tokens_exploded = df.tokens.explode().reset_index(drop=True)
labels_exploded = df.labels.explode().reset_index(drop=True)
labels_exploded.value_counts()

labels
O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

In [7]:
ids = labels_exploded[labels_exploded == 'B-STREET_ADDRESS'].index
print(" ".join(tokens_exploded[ids[0]-15:ids[0]+15]))
print("-"*50)
print(" ".join(tokens_exploded[ids[1]-15:ids[1]+15]))

and next time I want to concentrate on them , too 

 Waseem Mabunda   591 Smith Centers Apt . 656 
 Joshuamouth , RI 95963 ( The Netherlands )
--------------------------------------------------
the business model , and got ourselves an   identity ( 1861 . Milano - 743 Erika Bypass Apt . 419 
 Andreahaven , IL 54207 ) looking forward to


In [8]:
ids = labels_exploded[labels_exploded == 'I-STREET_ADDRESS'].index
tokens_exploded[ids]

861200           Smith
861201         Centers
861202             Apt
861203               .
861204             656
861205              \n
861206     Joshuamouth
861207               ,
861208              RI
861209           95963
1445331          Erika
1445332         Bypass
1445333            Apt
1445334              .
1445335            419
1445336             \n
1445337    Andreahaven
1445338              ,
1445339             IL
1445340          54207
Name: tokens, dtype: object

## Model Definition
We will attempt to create the simplest model possible. The model will evaluate each token and pass it through a couple functions to evaluate it. If one of the functions returns true, the token will be considered a keyword. The model will then return the keyword and the index of the token in the input string.

### Model Functions
- **email** This function will use regex to identify if the token is an email address.
- **phone_num** This function will use regex to identify if the token is a phone number.
- **address** This function will use regex to identify if the token is an address.
- **username** This function will use a word embedding model to identify if the token is a common word and if not then we flag it as a username.
- **personal_id** This function will use regex to identify if the token is a personal id.


In [9]:
names = NameDataset()

**To do**
Maybe we could make some more features for capitalised words etc.

In [10]:
names.first_names.get("Learning")

In [48]:
import re

def tokenizer(text):
    # Remove excessive line breaks or spaces
    text = re.sub('(\r\n){2,}', ' ', text)
    
    # Convert the text to lowercase and split into tokens
    tokens = text.lower().split(" ")
    
    return tokens

class PiiModel:
    def __init__(self, text: str = None, names_dataset = None, tokens: list = None, significant_tokens: dict = None, dataframe: pd.DataFrame = None) -> None:
        self.text = text
        self.names = names_dataset
        self.tokens = tokens
        self.tokenizer = tokenizer
        self.name_rank_threhsold = 1500
        self.country_threshold = 0.01
        self.significant_tokens = significant_tokens
        self.feature_df = pd.DataFrame()
        self.input_df = dataframe
        pass
    
    def get_firstname(self, name: str) -> bool:
        found_name = self.names.first_names.get(name.capitalize())
        # we check if US exists found_name.country
        if not found_name:
            return False
        country = found_name["country"]
        if country.get("US", 0) > self.country_threshold and found_name["rank"].get("US", 1e6) < self.name_rank_threhsold:
            return True
        return False

    
    def get_lastname(self, lastname: str) -> bool:
        found_name = self.names.last_names.get(lastname.capitalize())
        # we check if the max country is higher than a threshold then we say it is a valid name
        if not found_name:
            return False
        country = found_name["country"]
        if country.get("US", 0) > self.country_threshold and found_name["rank"].get("US", 1e6) < self.name_rank_threhsold:
            return True
        return False
    
    def get_email_regex(self):
        return r'[\w\.-]+@[\w\.-]+'
    
    def get_phone_number_regex(self):
        return r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b'
    
    def get_address_regex(self):
        return r'\d{1,5}\s\w.\s(\b\w*\b\s){1,2}\w*\.'
    
    def get_personal_id_regex(self):
        return r'\d{6,12}\b|\b[a-zA-Z]+\d{2,}'
    
    def get_url_personal_regex(self):
        return r'https?://[\w\.-]+'
    
    def numb_percent(self, text: str) -> float:
        return sum(c.isdigit() for c in text) / len(text)
    
    def word_length(self, text: str) -> int:
        return len(text)
    
    def is_punctuationmark(self, text: str) -> bool:
        return text in [".", ",", "!", "?", ";", ":"]
    
    def stop_words(self, text: str) -> bool:
        return text in nlp.Defaults.stop_words
    
    def NER_ingest(self, text: str):
        return nlp(text)
    
    def capitalize(self, text: str) -> bool:
        return text[0].strip().isupper()
    
    def contextual_regex(self, tokens: list, regex: str) -> bool:
        """ This function will attempt to incorporate the contextual information of the tokens to the regex.
        Essentially, we want to consider the tokens before and after the token to determine if the token is a match.
        Firstly, we check if the previous token is was detected as a match, if so, we add it to the current context, we do this until 
        we reach the beginning of the text or we find a token that was not detected as a match.
        Then, we move forward n steps and progressively add tokens and check if the regex matches the context.
        In the end, we return a boolean indicating if the regex was found in the context.

        Args:
            regex (str): The regex to be matched

        Returns:
            bool: A boolean indicating if the regex was found in the context
        """

        current_context = ""
        col_classifications = [False for _ in range(len(tokens))]
        n_forward = 3 # We move forward 3 steps

        # We commence a sliding window approach to check the context of the tokens
        for i in range(len(tokens)):
            # We iteratively move backwards until we reach the beginning of the text or we find a token that was not detected as a match
            while i >= 0 and col_classifications[i] == True:
                current_context = tokens[i] + " " + current_context
                i -= 1
            # We move forward n steps and progressively add tokens and check if the regex matches the context
            for j in range(n_forward):
                if i + j < len(tokens):
                    current_context = current_context + " " + tokens[i + j]
                    if re.match(regex, current_context):
                        col_classifications[i + j] = True
            current_context = ""
        return col_classifications

    
    def check_preceding_contextual_tokens(self, index, label):
        # Extract relevant tokens directly, avoiding repeated DataFrame row access
        start_index = max(0, index-2)
        tokens_to_check = self.feature_df['token'][start_index:index]
        
        # Get the set of significant tokens for the label, defaulting to an empty set if not found
        significant_tokens_set = self.significant_tokens.get(label, set())
        
        # Check if any of the tokens to check are in the significant tokens set
        return any(token in significant_tokens_set for token in tokens_to_check)

    
    def build_df(self, text: str = "", tokens: list = None, labels: list = None) -> pd.DataFrame:
        token_indices = self.tokens
        if not self.tokens and text != "":
            token_indices = self.tokenizer(text)
            self.tokens = token_indices
        if tokens:
            token_indices = tokens
        elif self.input_df is not None:
            token_indices = self.input_df["tokens"]
        self.feature_df["token"] = token_indices

        self.feature_df['labels'] = labels
        return None

    def build_features(self) -> list[tuple[int, str]]:
        # Computing Features
        self.feature_df['first_name'] = self.feature_df['token'].apply(lambda x: self.get_firstname(x))
        self.feature_df['last_name'] = self.feature_df['token'].apply(lambda x: self.get_lastname(x))
        self.feature_df['email'] = self.feature_df['token'].apply(lambda x: bool(re.match(self.get_email_regex(), x)))
        self.feature_df['r_neighboured_email'] = self.feature_df['email'].shift(-1).fillna(False)
        self.feature_df['l_neighboured_email'] = self.feature_df['email'].shift(1).fillna(False)
        self.feature_df['phone_number'] = self.contextual_regex(self.feature_df['token'], self.get_phone_number_regex())
        self.feature_df['r_neighboured_phone_number'] = self.feature_df['phone_number'].shift(-1).fillna(False)
        self.feature_df['l_neighboured_phone_number'] = self.feature_df['phone_number'].shift(1).fillna(False)
        self.feature_df['address'] = self.feature_df['token'].apply(lambda x: bool(re.match(self.get_address_regex(), x)))
        self.feature_df['r_neighboured_address'] = self.feature_df['address'].shift(-1).fillna(False)
        self.feature_df['l_neighboured_address'] = self.feature_df['address'].shift(1).fillna(False)
        self.feature_df['personal_id'] = self.feature_df['token'].apply(lambda x: bool(re.match(self.get_personal_id_regex(), x)))
        self.feature_df['r_neighboured_personal_id'] = self.feature_df['personal_id'].shift(-1).fillna(False)
        self.feature_df['l_neighboured_personal_id'] = self.feature_df['personal_id'].shift(1).fillna(False)
        self.feature_df['url'] = self.feature_df['token'].apply(lambda x: bool(re.match(self.get_url_personal_regex(), x)))
        self.feature_df['r_neighboured_url'] = self.feature_df['url'].shift(-1).fillna(False)
        self.feature_df['l_neighboured_url'] = self.feature_df['url'].shift(1).fillna(False)
        self.feature_df['capitalized'] = self.feature_df['token'].apply(lambda x: self.capitalize(x))
        self.feature_df['r_neighboured_capitalized'] = self.feature_df['capitalized'].shift(-1).fillna(False)
        self.feature_df['l_neighboured_capitalized'] = self.feature_df['capitalized'].shift(1).fillna(False)
        self.feature_df['numb_percent'] = self.feature_df['token'].apply(self.numb_percent)
        self.feature_df['r_neighboured_numb_percent'] = self.feature_df['numb_percent'].shift(-1).fillna(False)
        self.feature_df['l_neighboured_numb_percent'] = self.feature_df['numb_percent'].shift(1).fillna(False)
        self.feature_df['word_length'] = self.feature_df['token'].apply(self.word_length)
        self.feature_df['norm_index'] = self.feature_df.index / len(self.feature_df)
        self.feature_df['is_punctuation'] = self.feature_df['token'].apply(self.is_punctuationmark)
        self.feature_df['r_neighboured_punctuation'] = self.feature_df['is_punctuation'].shift(-1).fillna(False)
        self.feature_df['l_neighboured_punctuation'] = self.feature_df['is_punctuation'].shift(1).fillna(False)
        self.feature_df['stop_word'] = self.feature_df['token'].apply(self.stop_words)
        if self.input_df is not None:
            self.feature_df["text_index"] = self.input_df.text_index
        # Add a new feature for each label based on contextual tokens
        # for label in self.significant_tokens.keys():
        #     feature_name = f'{label.lower()}_contextual_presence'
        #     self.feature_df[feature_name] = [self.check_preceding_contextual_tokens(i, label) for i in range(len(self.feature_df))]


        return None


In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.tokens, df.labels, test_size=0.33, random_state=42)

In [50]:
dfs_analysis = []

for i in tqdm(range(X_train.shape[0])):
    tokens, labels = X_train.iloc[i], y_train.iloc[i]
    model = PiiModel(text=tokens, names_dataset=names)
    model.build_df(tokens=tokens, labels=labels)
    model.build_features()
    
    # Reset index and drop the old index to ensure unique indices
    model.feature_df.reset_index(drop=True, inplace=True)
    # drop duplicate columns
    model.feature_df = model.feature_df.loc[:,~model.feature_df.columns.duplicated()]
    
    # Append the DataFrame to the list
    dfs_analysis.append(model.feature_df)

X_analysis = pd.concat(dfs_analysis, axis=0)
X_analysis.fillna(0, inplace=True)


  0%|          | 0/4560 [00:00<?, ?it/s]

100%|██████████| 4560/4560 [01:06<00:00, 68.29it/s]


In [51]:
dfs_test = []

for i in tqdm(range(X_test.shape[0])):
    tokens, labels = X_test.iloc[i], y_test.iloc[i]
    model = PiiModel(text=tokens, names_dataset=names)
    model.build_df(tokens=tokens, labels=labels)
    model.build_features()
    
    # Reset index and drop the old index to ensure unique indices
    model.feature_df.reset_index(drop=True, inplace=True)
    # drop duplicate columns
    model.feature_df = model.feature_df.loc[:,~model.feature_df.columns.duplicated()]
    
    # Append the DataFrame to the list
    dfs_test.append(model.feature_df)

X_test = pd.concat(dfs_test, axis=0)
X_test.fillna(0, inplace=True)


100%|██████████| 2247/2247 [00:35<00:00, 63.44it/s]
